# Creación de base de datos para identificación de HVs

El presente archivo se creará la base de datos extrayendo información de HVs usando procesamiento de lenguaje natural. Esto con el objetivo de más adelante tener la posibilidad de generar con base en 300 HVs tanto rechazadas como avanzadas, un modelo que nos permita identificar las características predominantes de una HV que hace que avance o no durante el proceso.



#### Importar librerias

In [6]:
import os
import fitz  # PyMuPDF for PDFs
import pytesseract
from PIL import Image
import pandas as pd
import re
import spacy

### Importar SpaCy para NLP
Spacy es una librería de python que permite por medio de modelos de lenguaje pre-importados realizar análisis de texto, identificando palabras, nombres, lugares, objetos, verbos, adjetivos y la relación entre los mismos.

En este caso, importamos el modelo pre-entrenado en inglés, lo que requiere que todas las CVs a procesar estén en este idioma.


In [7]:
nlp = spacy.load("en_core_web_sm")

#### Cargar las carpetas con las HVs

In [8]:
hv_dir_exitosas = "HVs para base/Paso"
hv_dir_noexitosas = "HVs para base/No Paso"

### Deifinir palabras clave
En este caso, se definirar palabras clave que podrán tener las HVs teniendo en cuenta que para este modelo en particular se está utlizando solo HVs para un requerimiento de **desarrolladores Java**.

Debemos usar algunas repetidas para atenuar que hayan usado mayúsculas, espacios o símbolos


In [9]:
palabras_clave = ["Java", "java", "Spring", "spring", "spring boot", "Spring Boot", "AWS", "aws", "Azure", "azure", "GCP", "Google Cloud Platform", "google cloud platform", "microservices", "Maven", "maven", "gradle", "Gradle", "Java Server Pages", "JSP", "JEE", "Java Enterprise Edition", "Java8", "Java11", "Java17", "Java21", "JVM", "Java virtual machine", "Java Virtual Machine"]

### Definir las secciones y los patrones en las que estas van a aparecer
Además de definir la cantidad de palabras clave, es importante contar con las secciones con las que cada documento puede contar y entender si cuenta o no con este.

In [10]:
secciones = {
    "education": r"education|academic background|studies",
    "work_experience": r"work experience|employment history|professional experience|experience|professional background|background",
    "skills": r"skills|technical skills|competencies|programming languages|frameworks",
    "certifications": r"certifications|licenses|accreditations",
    "achievements": r"achievements|accomplishments|recognitions",
    "professional_profile": r"profile|summary|about me|professional summary|objective"
}

### Extraer el texto de los PDFs

A continuación se usará la librería FITZ, la cual ayuda a extraer el texto de un PDF, ver si tiene imágenes, contar sus páginas y detectar colores en los mismos.

La declaramos como función para llamarla más adelante en el procesamiento de todas las características que buscamos extraer.

In [11]:
def extraer_texto_pdf(pdf_path):
    text = ""
    try:
        doc = fitz.open(pdf_path)
        for page in doc:
            text += page.get_text("text") + "\n"
    except Exception as e:
        print(f"Leyendo PDF {pdf_path}: {e}")
    return text.strip()

### Contar palabras en general

In [12]:
def contar_palabras(text):
    return len(text.split()) if text else 0

#### Contar palabras clave

In [13]:
def contar_palabras_clave(text, keyword_list):
    text_lower = text.lower()
    return sum(1 for keyword in keyword_list if keyword.lower() in text_lower)

#### Extraer las secciones

Para extraer las secciones, usamos expresiones regulares. Con la biblioteca Re, busca el patron definido en la variable secciones más arriba, que ayuda a identificar si el texto obtenido del PDF tiene o no esta sección.

In [14]:
import re

def extraer_secciones(text):
    sections = {key: {"exists": False, "word_count": 0} for key in secciones.keys()}

    for section, pattern in secciones.items():
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            sections[section]["exists"] = True  # Section exists
            section_start = match.start()
            next_match = min(
                (m.start() for s, p in secciones.items() if (m := re.search(p, text[section_start + 1:], re.IGNORECASE))),
                default=len(text)
            )
            sections[section]["word_count"] = contar_palabras(text[section_start:section_start + next_match])

    return sections

### Verificar factores como foto y colores
De vuelta se usa la librería fitz para poder leer el PDF

#### Verificar si tiene o no foto

In [15]:
def tiene_foto_pdf(pdf_path):
    try:
        doc = fitz.open(pdf_path)
        for page in doc:
            if len(page.get_images(full=True)) > 0:
                return True
    except Exception as e:
        print(f"Error revisando foto en PDF {pdf_path}: {e}")
    return False

#### Verificar si tiene colores adicionales el PDF

In [16]:
def tiene_color_pdf(pdf_path):
    doc = fitz.open(pdf_path)

    for page in doc:
        for draw in page.get_drawings():
            if "color" in draw:
                return True

    return False

#### Contar páginas

In [17]:
def contar_paginas(pdf_path):
    try:
        doc = fitz.open(pdf_path)
        return len(doc)
    except Exception as e:
        print(f"Error counting pages in PDF {pdf_path}: {e}")
        return 1

### Procesamiento del CV
A continuación la función de procesamiento, nos ayudará a procesar un solo CV de acuerdo a los parámetros establecidos anteriormente, ejecutando cada una de las funciones ya establecidas

In [18]:
def process_cv(cv_path):
    text = ""
    has_photo = False
    has_colors = False
    num_pages = 1

    text = extraer_texto_pdf(cv_path)
    has_photo = tiene_foto_pdf(cv_path)
    has_colors = tiene_color_pdf(cv_path)
    num_pages = contar_paginas(cv_path)

    if not text:
        print(f"No se pudo extraer texto de {cv_path}")

    # Extract features
    total_word_count = contar_palabras(text)
    keyword_count = contar_palabras_clave(text, palabras_clave)
    sections = extraer_secciones(text)

    return {
        "CV_Name": os.path.basename(cv_path),
        "Total_Word_Count": total_word_count,
        "Has_Photo": int(has_photo),
        "Has_Colors": int(has_colors),
        "Pages": num_pages,
        "Keyword_Count": keyword_count,
        "Education_Exists": int(sections["education"]["exists"]),
        "Education_Word_Count": sections["education"]["word_count"],
        "Work_Experience_Exists": int(sections["work_experience"]["exists"]),
        "Work_Experience_Word_Count": sections["work_experience"]["word_count"],
        "Skills_Exists": int(sections["skills"]["exists"]),
        "Skills_Word_Count": sections["skills"]["word_count"],
        "Certifications_Exists": int(sections["certifications"]["exists"]),
        "Certifications_Word_Count": sections["certifications"]["word_count"],
        "Achievements_Exists": int(sections["achievements"]["exists"]),
        "Achievements_Word_Count": sections["achievements"]["word_count"],
        "Professional_Profile_Exists": int(sections["professional_profile"]["exists"]),
        "Professional_Profile_Word_Count": sections["professional_profile"]["word_count"],
    }

### Procesamiento de CVs en la carpeta
La siguiente función nos ayuda a de acuerdo con lo establecido anteriormente, procesar todas las CVs en las carpetas seleccionadas y devolverlas en una lista

In [19]:
def process_folder(folder_path, label):
    cv_data = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".pdf"):
            cv_path = os.path.join(folder_path, filename)
            print(f"Processing: {cv_path}")
            cv_info = process_cv(cv_path)
            cv_info["Passed"] = label
            cv_data.append(cv_info)
    return cv_data

## Creación de la base de datos

Se crean las variables donde se almacenan las CVs exitosas procesadas, agregando la información de 1 si es exitosa y 0 si no es exitosa.

In [ ]:
successful_data = process_folder(hv_dir_exitosas, 1)  # Label = 1 (Passed)
unsuccessful_data = process_folder(hv_dir_noexitosas, 0)  # Label = 0 (Not Passed)

Processing: HVs para base/Paso/HojaVida2022_CurriculumVitæ.pdf


Se guarda esta información en un dataframe

In [ ]:
data_total = successful_data + unsuccessful_data
baseCVs = pd.DataFrame(data_total)

In [ ]:
baseCVs

,CV_Name,Total_Word_Count,Has_Photo,Has_Colors,Pages,Keyword_Count,Education_Exists,Education_Word_Count,Work_Experience_Exists,Work_Experience_Word_Count,Skills_Exists,Skills_Word_Count,Certifications_Exists,Certifications_Word_Count,Achievements_Exists,Achievements_Word_Count,Professional_Profile_Exists,Professional_Profile_Word_Count,Passed
0,11686212-CV-Jorge Vidal.pdf,281,0,1,1,8,0,0,1,73,0,0,0,0,1,82,0,0,1
1,14483303-Andres_Gomez_resume.docx (2) (1).pdf,336,0,1,2,10,1,41,1,43,1,230,0,0,0,0,1,1,1
2,14503315-CV - Henry Luis Gomez Ortiz [En] (1) ...,760,0,1,2,8,1,5,1,5,1,29,0,0,0,0,1,25,1
3,14955159-Curriculum vitae english (1).pdf,767,1,1,3,11,1,118,1,22,1,12,0,0,0,0,1,29,1
4,8A9EBB5F-08ED-4D90-92C2-72A9024C2A58.pdf,482,0,1,2,4,0,0,1,38,0,0,0,0,1,387,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301,Sergio Resume (2).pdf,630,0,1,4,12,1,11,1,1,1,3,0,0,0,0,1,6,0
302,Sintesis_Curricular_Luis_Malpica_Murillo.pdf,763,1,1,3,11,0,0,0,0,1,182,0,0,0,0,0,0,0
303,Sucre - Specialist Fullstack Developer-3.pdf,1575,1,1,6,8,0,0,1,8,1,25,0,0,0,0,1,11,0
304,VictorPietro_Reis_Resume (1).pdf,372,0,1,2,8,1,2,0,0,1,125,0,0,0,0,0,0,0


### Exportar base en un archivo CSV para posterior lectura

In [ ]:
baseCVs.to_csv("baseCVs.csv", index=False)